# Tema 6: Agentes LLM + Tools + Skills — Teoría

## Ejercicio T1 - Tool vs Skill vs Agent

Define con tus palabras (sin copiar definiciones) las diferencias entre tool, skill y agente LLM. Incluye un ejemplo de cada uno en el contexto de un asistente universitario (normativa, tutorías, evaluación, etc.).

### Solución

| Concepto | Definición | Ejemplo (asistente universitario) |
|---|---|---|
| **Tool** | Acción atómica invocable con entrada/salida estructurada. | Función calculadora, consulta SQL, llamada a un retriever RAG. |
| **Skill** | Capacidad compuesta que encadena varias tools y pasos de razonamiento para lograr un subobjetivo reutilizable. | "Explicar una norma con ejemplo y advertencia" puede usar RAG + plantilla pedagógica. |
| **Agente** | Sistema que decide dinámicamente qué tool/skill ejecutar según la intención y el estado, manteniendo memoria y control de flujo. | Asistente de normativa que decide si necesita buscar documentos o responder directo. |

## Ejercicio T2 - Diseño de control: evitar loops y alucinaciones

Explica por qué los agentes pueden entrar en bucles o tomar acciones incorrectas. Propón 4 mecanismos concretos de control (p. ej., límite de pasos, validación de tool output, política de abstención, checks de groundedness) y di dónde se aplicarían en el pipeline.

### ¿Por qué ocurren los bucles y acciones incorrectas?

Los **bucles** ocurren cuando el agente no dispone de una condición de parada clara o cuando interpreta mal la observación de una tool y vuelve a intentar indefinidamente. Las **acciones incorrectas** aparecen por mala clasificación de intención, tool misuse o recuperación irrelevante.

### Mecanismos de control

| # | Mecanismo | Dónde se aplica |
|---|---|---|
| 1 | **Max steps / max tool calls**: cortar ejecución y devolver salida segura. | Control de flujo del agente (bucle principal). |
| 2 | **Validación de outputs**: schemas (Pydantic) o reglas (p.ej., calc output numérico) antes de pasar al LLM. | Entre la tool y el nodo de generación. |
| 3 | **Política de abstención**: si faltan evidencias (p.ej., contexto RAG vacío) responder "no puedo". | Nodo de generación de respuesta final. |
| 4 | **Guardrails de groundedness**: exigir citas/fuentes y verificar que la respuesta esté soportada por contexto. | Post-procesado de la respuesta generada. |
| 5 | **Reintentos controlados**: backoff + cambio de estrategia (MMR, query rewriting) con límite. | Nodo de retrieval / tool call. |

## Ejercicio T3 - RAG como tool dentro de un agente

Describe cómo integrarías un sistema RAG como tool dentro de un agente. ¿Qué decide el agente? ¿Qué devuelve la tool? ¿Qué riesgos aparecen (p. ej., retrieval irrelevante) y cómo los mitigarías?

### Integración

El **agente** decide "necesito evidencia documental" (router). Si sí, llama a una **tool RAG** (retriever + formateo) que devuelve contexto y fuentes. El LLM genera una respuesta *grounded* usando ese contexto.

### Riesgos y mitigaciones

| Riesgo | Mitigación |
|---|---|
| Retrieval irrelevante | MMR, filtros por metadata, query rewriting. |
| Chunks incompletos | Ajustar chunk_size/overlap, re-ranking. |
| Contradicciones entre chunks | Prompts que obliguen a citar y reconocer insuficiencia de contexto. |